# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:
import numpy as np
import torch
import torchvision
import tqdm
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/Dataset/dataset.zip' -d '/content'

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/dataset/dataset/val/class_101/00048.jpg  
  inflating: /content/dataset/dataset/val/class_101/00049.jpg  
   creating: /content/dataset/dataset/val/class_102/
  inflating: /content/dataset/dataset/val/class_102/00000.jpg  
  inflating: /content/dataset/dataset/val/class_102/00001.jpg  
  inflating: /content/dataset/dataset/val/class_102/00002.jpg  
  inflating: /content/dataset/dataset/val/class_102/00003.jpg  
  inflating: /content/dataset/dataset/val/class_102/00004.jpg  
  inflating: /content/dataset/dataset/val/class_102/00005.jpg  
  inflating: /content/dataset/dataset/val/class_102/00006.jpg  
  inflating: /content/dataset/dataset/val/class_102/00007.jpg  
  inflating: /content/dataset/dataset/val/class_102/00008.jpg  
  inflating: /content/dataset/dataset/val/class_102/00009.jpg  
  inflating: /content/dataset/dataset/val/class_102/00010.jpg  
  inflating: /content/dataset/dataset/val/class

### Подготовка данных

In [5]:
from torchvision.transforms import Compose, Normalize, Resize, ToTensor, RandomCrop, AutoAugment
train_transform = Compose(
        [
            AutoAugment(),
            ToTensor(), 
            Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
        ]
    )

val_transform = Compose(
        [
            ToTensor(), 
            Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
        ]
    )

train_dataset = torchvision.datasets.ImageFolder("./dataset/dataset/train", transform=train_transform)
val_dataset = torchvision.datasets.ImageFolder("./dataset/dataset/val", transform=val_transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= 256, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size= 256, shuffle = False, drop_last= False)

In [6]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [7]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    model = model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = np.array([])
    total_labels = np.array([])
    total_predictions = np.array([])

    for images, labels in tqdm(train_dataloader):
      images = images.to(device)
      labels = labels.to(device)
      predicted = model(images)
      loss = criterion(predicted, labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
      total_labels = np.append(total_labels, labels.cpu().detach().numpy())
      total_loss += loss.item()
      num_batches += 1
    scheduler.step()

    print(f"Train loss: {total_loss / num_batches}")
    print(f"Train accuracy: {(total_predictions == total_labels).mean()}")

def predict(model, val_dataloder, criterion, device="cuda:0"):
    model = model.to(device).eval()
    total_loss = 0
    total_labels = []
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])

    with torch.no_grad():
      for images, labels in val_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        predicted = model(images)
        loss = criterion(predicted, labels)
        accuracy = (predicted.argmax(1) == labels).float().mean()
        total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
        total_labels = np.append(total_labels, labels.cpu().detach().numpy())
        total_loss += loss.item()
        num_batches += 1
    
      print(f"Val loss: {total_loss / num_batches}")
      print(f"Val accuracy: {(total_predictions == total_labels).mean()}")
      losses = total_loss / num_batches
      predicted_classes = total_predictions
      true_classes = total_labels

    return losses, predicted_classes, true_classes


def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    model.to(device)
    for epoch in range(n_epochs):
        print(f"Epoch number {epoch}")
        train_one_epoch(model, train_dataloader, criterion, optimizer, device)
        all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
        if((predicted_labels == true_labels).mean() > 0.33):
          print("Train success!")
          break

### Обучение модели, запуски экспериментов

In [8]:
model = torchvision.models.vgg11(pretrained=False)

model.classifier[3] = nn.Linear(4096, 2048, bias= True)
model.classifier[6] = nn.Linear(2048, 200, bias= True)
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4, weight_decay=  1e-6)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma = 0.1)
criterion = nn.CrossEntropyLoss()
n_epochs = 70
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [9]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

Простой тест на проверку правильности написанного кода

In [10]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

Val loss: 5.29915943145752
Val accuracy: 0.0064
tests passed


In [18]:
import random
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

Запустить обучение можно в ячейке ниже.

In [28]:
set_random_seed(41)
train(model, train_dataloader, val_dataloader, criterion, optimizer, device, n_epochs, scheduler)

Epoch number 0


  0%|          | 0/391 [00:00<?, ?it/s]

Train loss: 5.218231030437343
Train accuracy: 0.00807
Val loss: 5.06577867269516
Val accuracy: 0.0118
Epoch number 1


  0%|          | 0/391 [00:00<?, ?it/s]

Train loss: 4.9719156304283825
Train accuracy: 0.02085
Val loss: 4.703585731983185
Val accuracy: 0.0346
Epoch number 2


...

Train loss: 2.7026681101230707
Train accuracy: 0.34634
Val loss: 2.8631593346595765
Val accuracy: 0.3223
Epoch number 13


  0%|          | 0/391 [00:00<?, ?it/s]

Train loss: 2.6641575869391945
Train accuracy: 0.35352
Val loss: 2.8452202677726746
Val accuracy: 0.3264
Epoch number 14


  0%|          | 0/391 [00:00<?, ?it/s]

Train loss: 2.6264566088576453
Train accuracy: 0.36067
Val loss: 2.8465073227882387
Val accuracy: 0.3303
Train success!


### Проверка полученной accuracy

После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, реализуйте и запустите функцию `evaluate`. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

In [29]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за это задание составит {} баллов".format(min(10, 10 * accuracy / 0.44)))

Val loss: 2.8465073227882387
Val accuracy: 0.3303
Оценка за это задание составит 7.506818181818182 баллов


### Отчёт об экспериментах 
Сначала я пытался сделать Transfer Learning для второй задачи с моделью resnet(18,34,50), но добился лишь 0,65 accuracy и забил на это задание.

Первое задание было намного ЛЕГЧЕ чем второе)

При проведении первого эксперимента я взял непредобученную модель из torchvision.models модель AlexNet при этом я не менял в модели ничего, кроме количества выходов модели, так как их изначально 1000(классов Imaginet), но в данной задаче их всего 200. При этом модель училась не так быстро как показанная в задании в конечной версии. AlexNet медленно доходила до accuracy 0,24 и далее градиент модели затухал. Качество модели на тестовой выборке не расло, а на обучающей возрастало, значит модель начала свое переобучение несмотря на аугментацию. Во избежании этого я добавил weight_decay, но ощутимого эффекта не наблюдалось.
После разочаровывания в Alexnet я посмотрел модели и их качество на сайте https://pytorch.org/vision/stable/models.html. После Alexnet была представлена vgg11, которую я использовал. На imaginet vgg показывала accuracy на 13% больше, чем AlexNet. Я использовал именно её, но изменив количество нейронов в нескольких полносвязных слоях, так как выходов было всего 200, а у полносвязных слоев наблюдалось количество нейронов около 4096, я сократил это число дол 2048. Модель обучалась быстро и обгоняла Alexnet на 8 эпохе, хотя Alexnet требовал около 20 эпох для такого качества. Эту модель я и оставил как результат задания.
Хочется упомянуть опыты с аугментацией, при выполнении 2 задания я пытался разными способами сделать аугментацию. Я использовал RandomCrop и RandomHorizontalFlip, что оказалось хуже по качеству, чем AutoAugment. Во втором задании при использовании AutoAugment качество было 0,6 accuracy, а при использовании первого варианта около 0,55. 
Также иногда обучение вело себя очень странно, модель в первом задании останавливалась на accuracy около 0,005 на валидиции и не становилось больше. После чего приходилось менять random_seed или перезапускать colab и модель опять обучалась.